In [13]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/mohitgupta-1O1/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", encoding='ISO-8859-1')[["v1", "v2"]].rename(columns={"v1": "label", "v2": "text"})

df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["label"])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df["text"].values, y, test_size=0.3, random_state=42, stratify=df["label"])

In [17]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(X_train)

X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [18]:
X_train[0]

[43,
 293,
 231,
 21,
 35,
 4,
 669,
 48,
 222,
 48,
 4,
 792,
 74,
 2,
 134,
 4,
 21,
 166,
 263]

In [19]:
X_train = pad_sequences(X_train, padding="post", maxlen=500)
X_test = pad_sequences(X_test, padding="post", maxlen=500)

In [20]:
X_train[0]

array([ 43, 293, 231,  21,  35,   4, 669,  48, 222,  48,   4, 792,  74,
         2, 134,   4,  21, 166, 263,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [22]:
model = Sequential(name="spam_detector")

model.add(Embedding(input_dim=len(token.word_index), output_dim=50, input_length=500))
model.add(Flatten())

model.add(Dense(10, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "spam_detector"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           370450    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                250010    
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 620471 (2.37 MB)
Trainable params: 620471 (2.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/100
390/390 [==============================] - 5s 10ms/step - loss: 0.0841 - accuracy: 0.9136 - val_loss: 0.0415 - val_accuracy: 0.9683
Epoch 2/100
390/390 [==============================] - 5s 13ms/step - loss: 0.0350 - accuracy: 0.9846 - val_loss: 0.0315 - val_accuracy: 0.9821
Epoch 3/100
390/390 [==============================] - 4s 9ms/step - loss: 0.0258 - accuracy: 0.9908 - val_loss: 0.0300 - val_accuracy: 0.9773
Epoch 4/100
390/390 [==============================] - 4s 9ms/step - loss: 0.0199 - accuracy: 0.9921 - val_loss: 0.0300 - val_accuracy: 0.9749
Epoch 5/100
390/390 [==============================] - 5s 12ms/step - loss: 0.0153 - accuracy: 0.9936 - val_loss: 0.0207 - val_accuracy: 0.9839
Epoch 6/100
390/390 [==============================] - 4s 10ms/step - loss: 0.0121 - accuracy: 0.9951 - val_loss: 0.0220 - val_accuracy: 0.9797
Epoch 7/100
390/390 [==============================] - 4s 10ms/step - loss: 0.0096 - accuracy: 0.9964 - val_loss: 0.0188 - val_accuracy: 0

In [24]:
loss, accuracy = model.evaluate(X_test, y_test)

print(loss)
print(accuracy)

53/53 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.9844
0.015408509410917759
0.9844497442245483
